In [1]:
import numpy as np 

# Cellular sheaves on graphs 
## Learning sheaf laplacian through minimum total variation approach 

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,2),
    (0,6),
    (1,3),
    (1,5),
    (2,3),
    (2,4),
    (3,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

In [3]:
d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(3,3),
        e[1]:np.random.randn(3,3)
        } 
        for e in edges
    }                                           # Incidency linear maps

In [4]:
F

{(0,
  1): {0: array([[ 1.21678325, -0.64968852, -0.89460929],
         [-3.18733149, -1.67188145, -0.08584367],
         [ 0.11322492,  0.00534064, -0.33607683]]), 1: array([[-0.54831161,  0.95712899, -0.08301372],
         [-1.00761323,  0.79804555,  0.66314696],
         [ 1.36891614,  0.0423488 , -1.42863907]])},
 (0,
  2): {0: array([[ 1.58838172,  0.95274986,  0.81571768],
         [-1.41957946, -0.04185778, -1.60963973],
         [ 0.79074478,  0.87039357, -0.36751744]]), 2: array([[ 0.18581487,  0.02174323,  2.01214212],
         [ 1.74958588,  0.32532705,  1.09229305],
         [-0.0460918 , -0.19860488,  0.42712734]])},
 (0,
  6): {0: array([[-0.79727272,  1.45662117,  0.19796459],
         [-0.46521848,  0.48730073, -1.56617731],
         [-0.60190199, -1.20749363,  0.38401967]]), 6: array([[ 2.15843929,  0.92997811, -0.32885673],
         [-0.68587106, -1.52826993,  0.9536552 ],
         [ 0.22810125, -1.57977923,  0.94871389]])},
 (1,
  3): {1: array([[-0.57374666,  1.1665

In [5]:
# Graph representation

A = np.zeros((7,7))

for edge in edges:
    u = edge[0] 
    v = edge[1] 

    A[u,v] = 1
    A[v,u] = 1

D = np.diag(np.sum(A, axis = 0))
L = D - A

In [6]:
# Sheaf representation 

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f = B.T @ B

In [7]:
B

array([[ 1.21678325, -0.64968852, -0.89460929,  0.54831161, -0.95712899,
         0.08301372,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-3.18733149, -1.67188145, -0.08584367,  1.00761323, -0.79804555,
        -0.66314696,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.11322492,  0.00534064, -0.33607683, -1.36891614, -0.0423488 ,
         1.42863907,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.58838172,  0.95274986,  0.81571768,  0.        

In [8]:
# Let's define a dataset through a diffusion process according to the sheaf laplacian 

N = 100
X = np.zeros((V*d,N))

In [9]:
X[0:3,:] = np.random.randn(3,N)

In [10]:
eigs, _ = np.linalg.eig(L_f)
eps = 2 / np.max(eigs)

In [11]:
X = ((np.eye(L_f.shape[0]) - eps*L_f)**3 @ X)

In [12]:
data = {node:X[node*d:(node+1)*d,:] for node in range(7)}

In [13]:
X_0 = data[0]
X_1 = data[1]

In [33]:
# Alternated Linear Maps Learning

def ALML(X_u, X_v, d, T = 50):
    # Initialization 

    F_u = np.eye(d)
    F_v = np.eye(d)
    gamma = 0.99

    # This matrices can be computed out of the learning loop 

    vu = X_v @ X_u.T
    uv = X_u @ X_v.T
    uu = np.linalg.inv(X_u @ X_u.T)
    vv = np.linalg.inv(X_v @ X_v.T)

    # Alternated learning through blockwise convex programs

    for _ in range(T): 
        # Local step
        F_u_hat = F_v @ vu @ uu
        F_v_hat = F_u @ uv @ vv

        # Convex smoothing
        F_u = F_u + gamma*(F_u_hat - F_u)
        F_v = F_v + gamma*(F_v_hat - F_v)

        gamma *= 0.9
    return F_u, F_v 

In [42]:
Fu, Fv = ALML(X_0, X_1, 3, T = 5)

In [44]:
Fv

array([[-1.87050062e+00,  1.37196335e+01,  9.20006841e-03],
       [ 1.01876474e+01,  3.98726487e+00,  2.33983948e+01],
       [-1.59431406e+02, -1.50431081e+02, -2.03266118e+02]])